In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pywaffle import Waffle

df = pd.read_csv("EER_2022_generation.csv")

gen = "generation_twh"
start = 2010
stop = 2021
years = [year for year in range(start,stop+1)]


In [ ]:
def change_dtype(x):
    try:
        return float(x)
    except:
        return x

icons = pd.read_json("icons.json").columns.tolist()
with open("icons.txt", mode='w') as file:
    file.write("\n".join(icons))

countries = df['country_name'].unique().tolist()

In [ ]:
icon_map = {
    'Fossils': 'gas-pump',
    'Solar': 'sun',
    'Wind': 'wind',
    'Nuclear': 'atom',
    'Hydro': 'water',
    'Bioenergy': 'leaf',
    'Other': 'temperature-high'
}

color_map = {
    'Fossils': '#7f7f7f',
    'Solar': '#ff7f0e',
    'Wind': '#17becf',
    'Nuclear': '#c5b0d5',
    'Hydro': '#1f77b4',
    'Bioenergy': '#2ca02c',
    'Other': '#d62728'
}


gas = '#8c564b'
oil = '#7f7f7f'
other_fossils = '#c7c7c7'
heat_pumps = '#c5b0d5'
direct_re = '#2ca02c'
district_heat = '#d62728'
electricity = '#1f77b4'
non_re_elec = '#ffbb78'
re_elec = '#98df8a'
re_DHC = '#d62728'
ambient = '#17becf'

""" 
FOSSILS 
- gas-pump
- oil-can
- biohazard
- spider
- poo-storm
- heart-broken

OTHERRENEW
- temperature-high
- pepper-hot
- mountain

"""

In [ ]:
df = df.apply(lambda x: change_dtype(x))
# df = df.set_index("fuel_code")

In [ ]:
def filter_by_country(target_country, df_func):
    filt = df_func["country_name"] == target_country
    return filt

def filter_by_year(target_year, df_func):
    filt_year = df_func["year"] == target_year
    return filt_year

In [ ]:
def create_yearly_gen_dict(df_country, years):
    x = {year: df_country[filter_by_year(year, df_country)][gen].to_dict() for year in years}
    return pd.DataFrame(x).T

def create_country_dict(dataframe, country):
    df = dataframe[filter_by_country(country, dataframe)]
    df.set_index("fuel_code",inplace=True)
    return df

def create_categories(df):
    df.fillna(0, inplace=True)
    df['Fossils'] = df['HARDCOAL'] + df['LIGNITE'] + df['OTHFOSS'] + df['GAS']
    df.drop(['HARDCOAL', 'LIGNITE','OTHFOSS','GAS'], axis=1, inplace=True)
    df.rename(columns={
    'SOLAR': 'Solar',
    'WIND': 'Wind',
    'NUCLEAR': 'Nuclear',
    'HYDRO': 'Hydro',
    'BIO': 'Bioenergy',
    'OTHRENEW': 'Other'}, 
    inplace=True)
    return df

def normalize(df):
    tot = df.sum(axis=1)
    for col in df:
        df[col] /= (tot)
        df[col] = round(df[col]*100,2)
        df[col] - round(df[col],0)
        #df[col] = df[col].astype(int)
    df.fillna(0, inplace=True)
    return df

In [ ]:
create_categories(create_yearly_gen_dict(create_country_dict(df, 'Germany'), years))

In [ ]:
def create_normalized_df(df, country):
    country_df = create_country_dict(df, country)
    gen_df = create_yearly_gen_dict(country_df, years)
    cat_df = create_categories(gen_df)
    plot_df = normalize(cat_df)
    return plot_df

In [ ]:
test = create_normalized_df(df, 'Germany')
test["total"] = test.sum(axis=1)
test

## WAFFLE CHART

In [ ]:
def plot_frame(dataframe, country, year):
    df_plot = create_normalized_df(dataframe, country)
    data = df_plot.loc[year].sort_values(ascending=False)
    labels = data.index.tolist()
    icons_plot = [icon_map.get(label) for label in labels]
    colors_plot = [color_map.get(label) for label in labels]
    fig = plt.figure(
        FigureClass=Waffle, 
        rows=10, 
        values=data,

        title={
            'label': f'{country} in {year}: Electricity Production',
            'loc': 'center',
            'fontdict': {
                'fontsize': 30, 
                'fontname': 'Microsoft Yi Baiti'
            }
        },
        legend={
            'labels' : labels,
            'loc': 8,
            'bbox_to_anchor': (0.5, -0.1),
            'ncol': len(data),
            'framealpha': 0,
            'fontsize': 12, 
        },
        icons = icons_plot,
        colors= colors_plot,
        dpi = 200,
        icon_size=35,
        icon_legend=True,
        block_arranging_style='snake',
        figsize=(12,12),
        facecolor='snow'
    )
    plt.text(0, -0.12, '@duncanmgibb', fontsize=15, fontfamily='Microsoft Yi Baiti', color='#7f7f7f')
    plt.savefig(f"exported/{country}_{year}")
    plt.show()

In [ ]:
create_normalized_df(df, 'Germany')

In [ ]:
plot_frame(df, 'Germany', 2021)

In [ ]:
for country in countries:
    plot_frame(df, country, 2021)

In [ ]:
def plot_bars(df):
    df.plot(kind='bar', stacked=True, figsize=(12,8), edgecolor='black')
    plt.legend(bbox_to_anchor=(1.15, 0.5), loc = 9, ncol=1, framealpha=0)
    plt.grid(alpha=0.4, axis='y', zorder=5)
    plt.title('Greece', y =1.08)
    #plt.xlabel(xlabel='', fontdict=text_props, labelpad=20)
    #plt.xticks(rotation=20, fontfamily=font, fontsize=bar_font-2)
    #plt.ylabel('Share', fontfamily=font, fontsize=bar_font)
    #plt.yticks(fontfamily=font, fontsize=bar_font)
    plt.margins(y=0)
    plt.tight_layout()
